In [1]:
import os
import cv2 as cv
from PIL import Image
from torchvision import transforms
import numpy as np
from tqdm import tqdm
import copy
import json

# Extract frames
Extract every 4 frames from video and save in dataset_frames

In [4]:
def save_frame(video, image_file):
    cap = cv.VideoCapture(video)
    idx_frame = 0
    idx_save = 0
    while True:
        ret, frame = cap.read()
        if ret:
            if idx_frame % 4 == 0:
                img = Image.fromarray(frame)
                img.save(os.path.join(image_file, f'{idx_save}.jpg'))
                idx_save +=1
            idx_frame += 1
        else:
            break
    cap.release()

In [5]:
save_file = './dataset_frames'
dataset = '../full_dataset'
scenes = os.listdir(dataset)
for scene in tqdm(scenes):
    if not scene.startswith('.'):
        cams = os.listdir(os.path.join(dataset,scene))
        for cam in cams:
            if not cam.startswith('.'):
                video_path = os.path.join(dataset, scene, cam)
                image_path = os.path.join(save_file, scene, os.path.splitext(cam)[0])
                if not os.path.exists(image_path):
                    os.makedirs(image_path)
                save_frame(video_path, image_path)

100%|██████████| 25/25 [10:07<00:00, 24.31s/it]


# Labeling frames
labeling clips with falling moment do not contains static laydown  
each clip contains 9 frames if including end  
save as python dictionary

## human label for fall clip

In [2]:
fall_clips_dict_train = {
    'chute01/cam1':[(270,278)],
    'chute01/cam2':[(270,278)],
    'chute01/cam3':[(270,278)],
    'chute01/cam4':[(270,278)],
    'chute01/cam5':[(276,284)],
    'chute01/cam6':[(270,278)],
    'chute01/cam7':[(270,278)],
    'chute01/cam8':[(270,278)],

    'chute02/cam1':[(100,108)],
    'chute02/cam2':[(103,111)],
    'chute02/cam3':[(93,101)],
    'chute02/cam4':[(93,101)],
    'chute02/cam5':[(97,105)],
    'chute02/cam6':[(102,110)],
    'chute02/cam7':[(102,110)],
    'chute02/cam8':[(96,104)],
    
    'chute03/cam1':[(151,159)],
    'chute03/cam2':[(151,159)],
    'chute03/cam3':[(151,159)],
    'chute03/cam4':[(151,159)],
    'chute03/cam5':[(157,165)],
    'chute03/cam6':[(153,161)],
    'chute03/cam7':[(153,161)],
    'chute03/cam8':[(148,156)],
    
    'chute04/cam1':[(92,100),(167,175)],
    'chute04/cam2':[(92,100),(167,175)],
    'chute04/cam3':[(92,100),(170,178)],
    'chute04/cam4':[(71,79),(149,157)],
    'chute04/cam5':[(87,95),(166,174)],
    'chute04/cam6':[(93,101),(170,178)],
    'chute04/cam7':[(93,101),(169,177)],
    'chute04/cam8':[(87,95),(163,171)],
    
    'chute05/cam1':[(82,90)],
    'chute05/cam2':[(84,92)],
    'chute05/cam3':[(79,87)],
    'chute05/cam4':[(81,89)],
    'chute05/cam5':[(82,90)],
    'chute05/cam6':[(84,92)],
    'chute05/cam7':[(84,92)],
    'chute05/cam8':[(79,87)],
    
    'chute06/cam1':[(149,157)],
    'chute06/cam2':[(174,182)],
    'chute06/cam3':[(176,184)],
    'chute06/cam4':[(172,180)],
    'chute06/cam5':[(172,180)],
    'chute06/cam6':[(175,183)],
    'chute06/cam7':[(175,183)],
    'chute06/cam8':[(174,182)],
    
    'chute07/cam1':[(127,135),(211,219)],
    'chute07/cam2':[(124,132),(207,215)],
    'chute07/cam3':[(125,133),(208,216)],
    'chute07/cam4':[(121,129),(204,212)],
    'chute07/cam5':[(121,129),(204,212)],
    'chute07/cam6':[(125,133),(208,216)],
    'chute07/cam7':[(123,131),(208,216)],
    'chute07/cam8':[(125,133),(208,216)],
    
    'chute08/cam1':[(90,98)],
    'chute08/cam2':[(88,96)],
    'chute08/cam3':[(68,76)],
    'chute08/cam4':[(88,96)],
    'chute08/cam5':[(84,92)],
    'chute08/cam6':[(88,96)],
    'chute08/cam7':[(88,96)],
    'chute08/cam8':[(83,91)],
    
    'chute09/cam1':[(160,168)],
    'chute09/cam2':[(159,167)],
    'chute09/cam3':[(157,165)],
    'chute09/cam4':[(162,170)],
    'chute09/cam5':[(160,168)],
    'chute09/cam6':[(160,168)],
    'chute09/cam7':[(160,168)],
    'chute09/cam8':[(157,165)],
    
    'chute10/cam1':[(129,137)],
    'chute10/cam2':[(129,137)],
    'chute10/cam3':[(129,137)],
    'chute10/cam4':[(134,142)],
    'chute10/cam5':[(130,138)],
    'chute10/cam6':[(130,138)],
    'chute10/cam7':[(130,138)],
    'chute10/cam8':[(129,137)],
    
    'chute11/cam1':[(121,129)],
    'chute11/cam2':[(116,124)],
    'chute11/cam3':[(121,129)],
    'chute11/cam4':[(119,127)],
    'chute11/cam5':[(116,124)],
    'chute11/cam6':[(117,125)],
    'chute11/cam7':[(117,125)],
    'chute11/cam8':[(117,125)],
    
    'chute12/cam1':[(161,169)],
    'chute12/cam2':[(157,165)],
    'chute12/cam3':[(159,167)],
    'chute12/cam4':[(159,167)],
    'chute12/cam5':[(155,163)],
    'chute12/cam6':[(155,163)],
    'chute12/cam7':[(158,166)],
    'chute12/cam8':[(158,166)],
    
    'chute13/cam1':[(113,121),(212,220)],
    'chute13/cam2':[(218,226)],
    'chute13/cam3':[(210,218)],
    'chute13/cam4':[(113,121),(212,220)],
    'chute13/cam5':[(118,126),(215,223)],
    'chute13/cam6':[(118,126),(215,223)],
    'chute13/cam7':[(118,126),(216,224)],
    'chute13/cam8':[(118,126),(212,220)],
    
    'chute14/cam1':[(149,157),(260,268)],
    'chute14/cam2':[(257,265)],
    'chute14/cam3':[(258,266)],
    'chute14/cam5':[(145,153),(257,265)],
    'chute14/cam6':[(145,153),(257,265)],
    'chute14/cam7':[(140,148),(249,257)],
    'chute14/cam8':[(143,151),(252,260)],
    
    'chute15/cam1':[(128,136),(193,201)],
    'chute15/cam2':[(128,136),(193,201)],
    'chute15/cam3':[(128,136),(193,201)],
    'chute15/cam4':[(128,136),(193,201)],
    'chute15/cam5':[(132,140),(196,204)],
    'chute15/cam6':[(132,140),(198,206)],
    'chute15/cam7':[(129,137),(193,201)],
    'chute15/cam8':[(125,133),(188,196)],
    
    'chute16/cam1':[(228,236)],
    'chute16/cam2':[(228,236)],
    'chute16/cam3':[(225,233)],
    'chute16/cam4':[(223,231)],
    'chute16/cam5':[(226,234)],
    'chute16/cam6':[(224,232)],
    'chute16/cam7':[(224,232)],
    'chute16/cam8':[(224,232)],
    
    'chute17/cam1':[(185,193)],
    'chute17/cam2':[(185,193)],
    'chute17/cam3':[(185,193)],
    'chute17/cam4':[(183,191)],
    'chute17/cam5':[(181,189)],
    'chute17/cam6':[(181,189)],
    'chute17/cam7':[(187,195)],
    'chute17/cam8':[(186,194)],
    
    'chute18/cam1':[(167,175)],
    'chute18/cam2':[(169,177)],
    'chute18/cam3':[(164,172)],
    'chute18/cam4':[(143,151)],
    'chute18/cam5':[(163,171)],
    'chute18/cam6':[(170,178)],
    'chute18/cam7':[(170,178)],
    'chute18/cam8':[(163,171)],
    
    'chute19/cam1':[(132,140)],
    'chute19/cam2':[(132,140)],
    'chute19/cam3':[(132,140)],
    'chute19/cam4':[(132,140)],
    'chute19/cam5':[(126,134)],
    'chute19/cam6':[(133,141)],
    'chute19/cam7':[(126,134)],
    'chute19/cam8':[(132,140)],
    
    'chute20/cam1':[(144,152),(164,172)],
    'chute20/cam2':[(139,147),(161,169)],
    'chute20/cam3':[(139,147),(161,169)],
    'chute20/cam4':[(140,148),(161,169)],
    'chute20/cam5':[(140,148),(161,169)],
    'chute20/cam6':[(140,148),(161,169)],
    'chute20/cam7':[(140,148),(161,169)],
    'chute20/cam8':[(140,148),(161,169)],
}

In [4]:
with open('fall_label_for_train.json', 'w') as file:
    json.dump(fall_clips_dict_train, file)

In [3]:
fall_clips_dict_test = {
    'chute21/cam1':[(223,231)],
    'chute21/cam2':[(226,234)],
    'chute21/cam3':[(226,234)],
    'chute21/cam4':[(218,226)],
    'chute21/cam5':[(218,226)],
    'chute21/cam6':[(227,235)],
    'chute21/cam7':[(227,235)],
    'chute21/cam8':[(219,227)],
    
    'chute22/cam1':[(193,201)],
    'chute22/cam2':[(203,211)],
    'chute22/cam3':[(207,215)],
    'chute22/cam4':[(207,215)],
    'chute22/cam5':[(207,215)],
    'chute22/cam6':[(205,213)],
    'chute22/cam7':[(205,213)],
    'chute22/cam8':[(204,212)],
}

In [10]:
with open('fall_label_for_test.json', 'w') as file:
    json.dump(fall_clips_dict_test, file)

## Compute not fall clip
50 frames before fall clip

In [4]:
not_fall_clips_dict_train = copy.deepcopy(fall_clips_dict_train)
not_fall_clips_dict_train['chute20/cam1'] = [(144,152)]
not_fall_clips_dict_train['chute20/cam2'] = [(139,147)]
not_fall_clips_dict_train['chute20/cam3'] = [(139,147)]
not_fall_clips_dict_train['chute20/cam4'] = [(140,148)]
not_fall_clips_dict_train['chute20/cam5'] = [(140,148)]
not_fall_clips_dict_train['chute20/cam6'] = [(140,148)]
not_fall_clips_dict_train['chute20/cam7'] = [(140,148)]
not_fall_clips_dict_train['chute20/cam8'] = [(140,148)]
for scene, expands in not_fall_clips_dict_train.items():
    for idx in range(len(expands)):
        expands[idx] = (expands[idx][0]-60, expands[idx][1]-18)

In [7]:
with open('not_fall_label_for_train.json', 'w') as file:
    json.dump(not_fall_clips_dict_train, file)

In [8]:
not_fall_clips_dict_test = copy.deepcopy(fall_clips_dict_test)
for scene, expands in not_fall_clips_dict_test.items():
    for idx in range(len(expands)):
        expands[idx] = (expands[idx][0]-60, expands[idx][1]-18)

In [10]:
with open('not_fall_label_for_test.json', 'w') as file:
    json.dump(not_fall_clips_dict_test, file)

## Check the correctness of label

In [6]:
data_dir = './dataset_frames'
for scene, expands in fall_clips_dict_test.items():
    for expand in expands:
        if expand[1]-expand[0] != 8:
            print(scene)

# Generate img data

In [13]:
def save_img_seq(img_dir, start, end, save_dir):
    trans = transforms.Compose([transforms.CenterCrop(480),transforms.Resize(224)])
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    for idx in range(start, end):
        img = Image.open(os.path.join(img_dir,f'{idx}.jpg'))
        img = trans(img)
        img.save(os.path.join(save_dir,f'{idx}.jpg'))

In [17]:
img_folder = './dataset_frames'
save_folder = './not_fall_img_train'

for scene, expands in tqdm(not_fall_clips_dict_train.items()):
    img_dir = os.path.join(img_folder, scene)
    save_dir = os.path.join(save_folder, scene)
    for expand in expands:
        save_img_seq(img_dir, *expand, save_dir)

100%|██████████| 159/159 [01:17<00:00,  2.05it/s]


# Generate CSV

In [5]:
def get_img_list(data_dir, frame_dict):
    img_names = []
    for scene, expands in frame_dict.items():
        for cam in os.listdir(os.path.join(data_dir, scene)):
            if not cam.startswith('.'):
                img_dir = os.path.join(data_dir,scene,cam)
                for expand in expands:
                    for img_idx in range(*expand):
                        img_name = os.path.join(img_dir,f'{img_idx}.jpg')
                        img_names.append(img_name)
    return img_names

In [6]:
data_dir = './dataset_frames'
# get frame for fall and not fall
train_f_img = get_frame_list(data_dir, trainset_fall)
train_f_lbl = list(np.ones(len(train_f_img)))
train_n_img = get_frame_list(data_dir, trainset_not_fall)
train_n_lbl = list(np.zeros(len(train_n_img)))
# mix fall and not fall and save
train_f_img.extend(train_n_img)
train_f_lbl.extend(train_n_lbl)
train_pd = pd.DataFrame({'img':train_f_img, 'label':train_f_lbl})
train_pd.to_csv('train.csv')

In [7]:
# get frame for fall and not fall
test_f_img = get_frame_list(data_dir, testset_fall)
test_f_lbl = list(np.ones(len(test_f_img)))
test_n_img = get_frame_list(data_dir, testset_not_fall)
test_n_lbl = list(np.zeros(len(test_n_img)))
# mix fall and not fall and save
test_f_img.extend(test_n_img)
test_f_lbl.extend(test_n_lbl)
test_pd = pd.DataFrame({'img':test_f_img, 'label':test_f_lbl})
test_pd.to_csv('test.csv')